In [22]:
import numpy as np
import pandas as pd
import re

pd.options.display.max_columns = 100
idx = pd.IndexSlice

%matplotlib inline
import matplotlib.pyplot as pt
import seaborn as sb
sb.set_style('whitegrid')

import wikifunctions as wf

import requests, json, re, time, os, pprint

path = 'catlists\\'
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote, quote
from copy import deepcopy

In [2]:
#set up an empty cat-list pair legend
#df=pd.DataFrame(columns=['category','list','suffixes'])
#df.head()
#df.to_csv(path+'keyPairs.csv',index=False)

In [31]:
def get_list(listName):
    l=listName.replace(' ','_')
    #df=pd.DataFrame()
    r = requests.get('https://en.wikipedia.org/wiki/'+str(l))
    con = r.content
    #print(con)
    df = pd.read_html(con)
    df=df[0]
    #df.to_csv(path+l+'.csv')
    df['inList']=listName
    df['listLink']='https://en.wikipedia.org/wiki/'+str(l)
    return df

In [37]:
def standardizeLists(df):
    #name
    dz=pd.DataFrame(columns=['name'])
    try:
        dz['name']=df['Name']
    except:
        print(df.head())
        return 
    #birth and/or death years
    try:
        dz['LGBTQ Identity']=df['Sexual orientation and/or gender identity']
    except:
        try:
            dz['LGBTQ Identity']=df['Sexual orientation and/or gender identity[2]']
        except:
            pass    
    try:
        dz['years']=df['Dates']
    except:
        try:
            dz['years']=df['Birth year']
        except:
            try:
                dz['years']=df['Lifetime']
            except:
                try:
                    dz['years']=df['Lifetime[1]']
                except:
                    pass
    try:
        dz['years'].apply(lambda s:s.strip('b.'))
    except:
        pass
    #nationality
    try:
        dz['country']=df['Nationality']
    except:
        try:
            dz['country']=df['Country']
        except:
            pass
    #pronouns maybe
    try:
        dz['pronouns']=df['Pronouns']
    except:
        pass
    #misc info
    try:
        dz['misc']=df['Comments']
    except:
        try:
            dz['misc']=df['Notable as']
        except:
            try:
                dz['misc']=df['Notes']
            except:
                pass
    dz['inList']=df['inList']
    dz['listLink']=df['listLink']
    
    return dz

In [4]:
def get_catMembers(categoryName, d=3):
    c=categoryName.replace(' ','_')
    catMembers = wf.get_category_members('Category:'+c,
                                        depth=d)
    
    df=pd.DataFrame()
    df['name']=catMembers
    df['wikiLink']=df['name'].apply(lambda s: 'https://en.wikipedia.org/wiki/'+str(s.replace(',','').replace(' ','_')))
    df['inCategory']=categoryName
    return df

In [5]:
#d=get_list('List of transgender people')
#d=standardizeLists(d)
#d.head()

In [6]:
#this method was the last thing causing problems so I just pulled it apart in a loop later

def diffListCat(categoryName,listName,suffixes=[]):
    a=get_catMembers(categoryName)
    b=get_list(listName,suffixes)
    b=standardizeLists(b)
    try:
        c=list(set(b['name']) and set(a['name']))
        print(c)
    except:
        return
    notInList=b[b['name'] not in c]
    notInCategory=a[a['name'] not in c]
    notInList['addToList']=listName
    notInCategory['addToCategory']=categoryName
    return [notInList, notInCategory]

In [13]:
df=pd.read_csv(path+'keyPairs.csv',keep_default_na=False,index_col=False)
addToList=pd.DataFrame()
addToCat=pd.DataFrame()
for index, row in df.iterrows():
    try:
        ls,ct=diffListCat(categoryName=row[0],listName=row[1],suffixes=row[2])
        addToList.append(ls)
        addToCat.append(ct)
    except:
        pass

In [ ]:
addToList.to_csv('addToList.csv',index=False)
addToCat.to_csv('addToCat.csv',index=False)

In [33]:
#testing the get_list() method
ls=get_list('List of LGBT writers')
ls.head()

,Name,Lifetime,Nationality,Genre,Notable works,References,inList,listLink
0,Caio Fernando Abreu,1948–1996,Brazilian,"short stories, plays","Morangos Mofados, dragões não conhecem o paraíso",[1],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
1,Barry D. Adam,b. 1952,Canadian,scholar,"The Rise of a Gay and Lesbian Movement, The Gl...",[2],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
2,Evan Adams,b. 1966,Canadian,playwright,"Dreams of Sheep, Snapshots, Dirty Dog River an...",[3],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
3,Nathan Niigan Noodin Adler,b. ?,Canadian,horror,"Wrist, Ghost Lake",[4],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
4,Etel Adnan,b. 1925,Lebanese-American,poet,Sea and Fog,[5],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [35]:
#testing the standardizeLists() method
standardized_ls=standardizeLists(ls)
standardized_ls.head()

,name,years,country,inList,listLink
0,Caio Fernando Abreu,1948–1996,Brazilian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
1,Barry D. Adam,b. 1952,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
2,Evan Adams,b. 1966,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
3,Nathan Niigan Noodin Adler,b. ?,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
4,Etel Adnan,b. 1925,Lebanese-American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [36]:
cm=get_catMembers('LGBT writers')
cm.head()

,name,wikiLink,inCategory
0,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,LGBT writers
1,Rob Astbury,https://en.wikipedia.org/wiki/Rob_Astbury,LGBT writers
2,Anton Enus,https://en.wikipedia.org/wiki/Anton_Enus,LGBT writers
3,Narelda Jacobs,https://en.wikipedia.org/wiki/Narelda_Jacobs,LGBT writers
4,Patricia Karvelas,https://en.wikipedia.org/wiki/Patricia_Karvelas,LGBT writers


In [42]:
z=standardized_ls.merge(cm, on='name', how='inner').drop_duplicates()

In [47]:
notInCategory=standardized_ls[~standardized_ls.name.isin(cm['name'].unique())]
notInCategory.head()

,name,years,country,inList,listLink
6,Magaly Alabau,b. 1945,Cuban American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
15,Lisa Alther,b. 1944,American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
21,Patrick Anderson,1915–1979,English/Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
22,Albalucía Angel,b. 1939,Colombian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
23,Núria Añó,b. 1973,Spanish,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [48]:
notInList=cm[~cm.name.isin(standardized_ls['name'].unique())]
notInList.head()

,name,wikiLink,inCategory
0,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,LGBT writers
1,Rob Astbury,https://en.wikipedia.org/wiki/Rob_Astbury,LGBT writers
2,Anton Enus,https://en.wikipedia.org/wiki/Anton_Enus,LGBT writers
3,Narelda Jacobs,https://en.wikipedia.org/wiki/Narelda_Jacobs,LGBT writers
4,Patricia Karvelas,https://en.wikipedia.org/wiki/Patricia_Karvelas,LGBT writers


In [59]:
df=pd.read_csv(path+'keyPairs.csv',keep_default_na=False,index_col=False)
#addToList=pd.DataFrame()
#addToCat=pd.DataFrame()
addToList=pd.read_csv(path+'addToList.csv',keep_default_na=False)
addToCat=pd.read_csv(path+'addToCat.csv',keep_default_na=False)

In [60]:
df.head()

,category,list,suffixes
0,LGBT sportspeople,List of LGBT sportspeople,
1,LGBT African Americans,List of LGBT African Americans,
2,LGBT writers,List of LGBT writers,


In [65]:
for index, row in df.iterrows():
    listName=row['list']
    categoryName=row['category']
    ls=get_list(listName)
    #ls.head()
    standardized_ls=standardizeLists(ls)
    #standardized_ls.head()
    cm=get_catMembers(categoryName)
    #cm.head()
    notInCategory=standardized_ls[~standardized_ls.name.isin(cm['name'].unique())]
    notInCategory['addThisCategoryTag']=categoryName
    #notInCategory.head()
    notInList=cm[~cm.name.isin(standardized_ls['name'].unique())]
    notInList['addToThisList']=listName
    notInList['listLink']='https://en.wikipedia.org/wiki/'+str(listName.replace(' ','_'))
    #notInList.head()
    addToList=pd.concat([addToList,notInList],ignore_index=True)
    addToCat=pd.concat([addToCat,notInCategory],ignore_index=True)


<ipython-input-65-9560cef97c43>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notInList['addToThisList']=listName
<ipython-input-65-9560cef97c43>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notInList['listLink']='https://en.wikipedia.org/wiki/'+str(listName.replace(' ','_'))


In [66]:
addToList.tail()

,name,wikiLink,inCategory,addToThisList,listLink
38017,Charlyne Yi,https://en.wikipedia.org/wiki/Charlyne_Yi,LGBT writers,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
38018,Reuben Zellman,https://en.wikipedia.org/wiki/Reuben_Zellman,LGBT writers,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
38019,Xiran Jay Zhao,https://en.wikipedia.org/wiki/Xiran_Jay_Zhao,LGBT writers,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
38020,Nevo Zisin,https://en.wikipedia.org/wiki/Nevo_Zisin,LGBT writers,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
38021,Rachel Zolf,https://en.wikipedia.org/wiki/Rachel_Zolf,LGBT writers,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [67]:
addToCat.tail()

,name,LGBTQ Identity,years,country,inList,listLink,misc
376,Núria Añó,NaN,b. 1973,Spanish,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,NaN
377,Salvatore Antonio,NaN,b. 1976,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,NaN
378,Gloria Anzaldúa,NaN,1942–2004,Chicana,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,NaN
379,Rafael Arévalo Martínez,NaN,1884–1975,Guatemalan,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,NaN
380,Manuel Azaña,NaN,1880–1940,Spanish,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,NaN


In [70]:
addToList.to_csv(path+'addToList.csv',index=False)
addToCat.to_csv(path+'addToCat.csv',index=False)